In [6]:
import pandas as pd
import random
from pathlib import Path
import regex as re
import os

_SHUFFLING_ORDER = [17, 13, 7, 14, 8, 12, 18, 0, 2, 6, 5, 4, 11, 1, 19, 16, 15, 10, 3, 9]

In [ ]:
def template(txta, txtb):
    return f"""<p style="text-align: justify;"><span style="font-size: 14px;">Il testo B segue il testo A. Pensi che B sia stato generato automaticamente?<br><br><strong>A:</strong>&nbsp;{txta}</span></p><hr id="null"><span style="font-size: 16px;"><strong>B:&nbsp;</strong>{txtb}</span>"""

In [ ]:
control_txta = """Nel giro di poco, Facebook potrebbe potenzialmente raggiungere 400 milioni di iscritti al miliardo e 800 milioni attuali. Si tratta della popolazione della Cina , mercato che Mark Zuckenberg punta da tempo."""
control_txtb = """E' quello che ha dichiarato il fondatore di Facebook in un'intervista a Bloomberg. "Siamo pronti a fare tutto quello che è necessario per entrare in Cina" ha detto Zuckerberg. "Siamo pronti a fare tutto quello che è necessario per entrare in Cina". "Siamo pronti a fare tutto quello che è necessario per entrare in Cina". "Siamo pronti a fare tutto quello che è necessario per entrare in Cina". "Siamo pronti a fare..."""
control_q = template(control_txta, control_txtb)

In [ ]:
rephrased_df = pd.read_csv("./change-it.ilgiornale.test_1000_rephrased_epoch_00006.csv", sep="\t", index_col=0, lineterminator="\n")

In [ ]:
n_questionaries = 6
q_per_questionary = 20

In [ ]:
random.seed(42)
evaldf = pd.read_csv("./questions_import_sample_IT.csv")
dfs1 = []
dfs2 = []
for i in range(n_questionaries):
    df = rephrased_df.iloc[q_per_questionary * i: q_per_questionary * (i + 1), :]
    split1 = range(q_per_summary // 2)
    split2 = range(q_per_summary // 2, q_per_summary)
    future_df1 = evaldf.to_dict()
    for key in list(future_df1.keys()):
        future_df1[key] = [future_df1[key][0]]
    future_df2 = evaldf.to_dict()
    for key in list(future_df2.keys()):
        future_df2[key] = [future_df2[key][0]]
        
    future_df1["is_human"] = ["boh"]
    future_df2["is_human"] = ["boh"]
    
    for i in split1:
        for key in list(future_df1):
            if key not in ["Testo domanda", "is_human"]:
                future_df1[key].append(future_df1[key][-1])
                future_df2[key].append(future_df2[key][-1])
                continue
            if key == "Testo domanda":
                future_df1[key].append(
                    template(
                        df.iloc[i, :].prompts,
                        df.iloc[i, :].true_continuations
                    )
                )
                future_df2[key].append(
                    template(
                        df.iloc[i, :].prompts,
                        df.iloc[i, :].generated_continuations
                    )
                )
            elif key == "is_human":
                future_df1[key].append(True)
                future_df2[key].append(False)

                
    for i in split2:
        for key in list(future_df1):
            if key not in ["Testo domanda", "is_human"]:
                future_df1[key].append(future_df1[key][-1])
                future_df2[key].append(future_df2[key][-1])
                continue
            if key == "Testo domanda":
                future_df1[key].append(
                    template(
                        df.iloc[i, :].prompts,
                        df.iloc[i, :].generated_continuations
                    )
                )
                future_df2[key].append(
                    template(
                        df.iloc[i, :].prompts,
                        df.iloc[i, :].true_continuations
                    )
                )
            elif key == "is_human":
                future_df1[key].append(False)
                future_df2[key].append(True)

    # drop first
    for key in list(future_df1.keys()):
        future_df1[key] = future_df1[key][1:]
    for key in list(future_df2.keys()):
        future_df2[key] = future_df2[key][1:]
    
    shuffled_df1 = {}
    for key, val in future_df1.items():
        shuffled_df1[key] = [val[idx] for idx in _SHUFFLING_ORDER]
    shuffled_df2 = {}
    for key, val in future_df2.items():
        shuffled_df2[key] = [val[idx] for idx in _SHUFFLING_ORDER]
        
    dfs1.append(shuffled_df1)
    dfs2.append(shuffled_df2)
    # pd.DataFrame.from_dict(future_df1).to_csv("human_eval_df1.csv", encoding="utf-8")
    # pd.DataFrame.from_dict(future_df2).to_csv("human_eval_df2.csv", encoding="utf-8")

In [ ]:
# for i in pd.DataFrame.from_dict(future_df1).loc[:, "Testo domanda"].apply(lambda x: re.findall("""/strong>.*</span>""", x)[0]):
#     print(i)
#     print()
pd.DataFrame.from_dict(dfs1[2])